# CS152 ASSIGNMENT 2: THE DPLL ALGORITHM 
### Vinícius Miranda

The code below implements the DPLL algorithm to check the satisfiability of any given KB in CNF form. 

#### Relevant HCs:
- #optimization: The code is heavily optimized, both from a theoretical level due the application of all the pre-specified extension heuristics and from an implementation level, with the use of cacheing of class instances and effective list/set  comprehensions.

#### Decorators

I implement two decorators which will be useful in due time. `count_calls` will help us assess whether optimization heuristics applied to the DPLL algorithm decrease the number of recursion calls. `cache` will save us work in generating new classes. In formal logic, the negation of a negated literal is the literal itself (i.e., `--A = A`). Hence, instead of creating a new literal, we return the one which was previously created. This technical detail allows us to use the `is` operator in Python to compare literals, although the operator is not used in this implementation. Decorators are slightly modified versions of the ones shown in Hjelle (2018).

In [1]:
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

def cache(cls):
    """Keep a cache of previous function calls"""
    @functools.wraps(cls)
    def wrapper_cache(*args, **kwargs):
        # If sign is false, concatenate kwargs. If not, use just the name
        cache_key = args + tuple(kwargs.items()) if kwargs and not kwargs['sign'] else args
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = cls(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = dict()
    return wrapper_cache


In [2]:
@cache
class Literal:
    '''Represents a literal which may or may not be negated.
       Hash value and equality are represented different because we 
       want a negated symbol to be equal to itself, but not be found
       on the set which includes only its negated version.
    '''
    
    def __init__(self, name, sign = True):
        self.name = name
        self.sign = sign
        
    def __neg__(self):
        return(Literal(self.name, sign = not self.sign))
    
    def __repr__(self):
        repr_aux = "" if self.sign else "-"
        return(repr_aux + self.name)
    
    def __hash__(self):
        return(hash(self.__repr__()))
    
    def __eq__(self, other):
        return(self.name == other.name)
    
def test_Literal():
    '''Test suite for the Literal class.'''
    
    A, B, C = (Literal(_) for _ in  "ABC")
    assert -A == A, "Complementary literals do not refer to the same symbol"
    assert A != B, "Different symbols are said to be equal"
    assert (--A).sign == A.sign, "Composability of negation malfunctions"
    assert A.__repr__() == A.name, "The representation of a symbol is incorrect"
    assert (-B).__repr__() == "-" + B.name, "The representation of a negated literal is incorrect"
    assert A in {A}, "Hashing of symbols malfunctions"
    assert -A in {-A}, "Hashing of symbols malfunctions"
    assert A not in {-A}, "Hashing of symbols malfunctions"
    assert C not in {A, B}, "Hashing of symbols malfunctions"
    assert A is --A and id(A) == id(--A), "Class equality malfunctions"
    
test_Literal()

In [10]:
@count_calls
def DPLL(clauses, symbols, model, debug = False, optim = True):
    '''Internal DPLL subroutine. The subroutine differs slightly from Russell & Norvig.
       It guarantees that clauses is always a minimal representation of the CNF given what
       can be entailed from the model, i.e., (A v B) reduces to B if -A is in the model
       or to True if A is in the model. The routine first checks the base cases, then
       finds the best symbol P to assign. It then reduce the clauses and generates the 
       recursion cases. Optional arguments percolate through the recursion.
           
           Inputs:
               - clauses (list of sets of Literals): A propositional logic 
                   sentence in CNF form. E.g., [{A,B}, {-A,C}]
               - symbols (set of Literals): A set of (positive) symbols not yet assigned.
               - model (set of Literals): A set of Literals already assigned.
               - debug (bool): Show the inputs for every recursion call. Defaults to False.
               - optim (bool): Whether to use optimization heuristics.
       
           Outputs: Tuple of (bool, dict/set). The first value indicates whether 
               the model satisfies the sentence in clauses. The second value
               provides a dict of Literals and true/false/free value pairs, according
               to the assignments made during execution. If unsatisfiable, the second
               value is the empty set.
    '''
    if debug: print(clauses, symbols, model)
    
    # If the list of clauses is empty, all of them have been proved
    # Construct the final model based on what has been previously assigned
    if not clauses:
        final_model = {}
        for lit in symbols | model:
            key = lit if lit.sign else -lit
            if lit in model:
                final_model[key] = "true" if lit.sign else "false"
            else:
                final_model[key] = "free"
        return(True, final_model)
    
    # A sentence is unsatisfiable if it includes the empty set 
    # (i.e., a clause that can't be proved).
    if set() in clauses: return (False, set())
    
    if optim:
        # ------- EXTENSION 2a: PURE SYMBOL ----------
        # Check for the presence of each symbol in the clauses
        # Implementation could be improved as it is currently O(m*n) for m symbols and n clauses.
        symbol_presence = [(sym, [sym2.sign for cl in clauses for sym2 in cl if sym == sym2])
                           for sym in symbols]
        pure_symbols = [(sym, len(count)) if sum(count) > 0 else (-sym, len(count)) 
                                          for sym, count in symbol_presence 
                                          if sum(count) == 0 or sum(count)/len(count) == 1]
        if pure_symbols:
            P = max(pure_symbols, key = lambda x: x[1])[0]
            # Symbols only contains positive literals so if P is a pure negated symbol, 
            # remove its negation (i.e., the positive literal)
            rest = symbols - {P} if P.sign else symbols - {-P}
            min_clauses = [set(sym for sym in cl if P != sym) for cl in clauses if P not in cl]
            return(DPLL(min_clauses, rest, model | {P}, debug, optim))
        
        # ------- EXTENSION 2b: UNIT CLAUSE ----------
        try:
            P, = next(cl for cl in clauses if len(cl) == 1)
            rest = symbols - {P} if P.sign else symbols - {-P}
            min_clauses = [set(sym for sym in cl if P != sym) for cl in clauses if P not in cl]
            return(DPLL(min_clauses, rest, model | {P}, debug, optim))

        # if next() can't find a unit clause, it will raise an error. 
        except StopIteration:
            pass

    # ------- EXTENSION 1: DEGREE ----------------
    # Find the symbol that occurs in the most number of clauses. We first try pure symbols, then impure.
    # We count the number of occurences of each symbol, sum them, and find the maximum.
    P = max([(sym, sum(1 for cl in clauses for sym2 in cl if sym2 == sym)) for sym in symbols], 
            key = lambda x: x[1])[0] if optim else next(iter(symbols))
    rest = symbols - {P}
    
    # Reduce clauses.
    this_min_clauses = [set(sym for sym in cl if P != sym) for cl in clauses if P not in cl]
    that_min_clauses = [set(sym for sym in cl if P != sym) for cl in clauses if -P not in cl]

    # Because we're returning two objects, we construct if clauses for the recursion.
    # The or operator misbehaves when comparing tuples.
    this_recur = DPLL(this_min_clauses, rest, model | {P}, debug, optim)
    if this_recur[0]:
        return(this_recur)
    
    that_recur = DPLL(that_min_clauses, rest, model | {-P}, debug, optim)
    if that_recur[0]:
        return(that_recur)
    else:
        return(False, set())    

def DPLLSatisfiable(KB, debug=False, optim=True):
    '''Checks whether the KB is satisfiable by calling DPLL.
    
           Inputs:
               - KB (list of sets of Literals): A propositional logic 
                   sentence in CNF form. E.g., [{A,B}, {-A,C}]
               - debug (bool): Show the inputs for every recursion call. Defaults to False.
               - optim (bool): Whether to use optimization heuristics.
               
           Outputs: Tuple of (bool, dict/set). Same as in DPLL, above.
    
    '''
    
    # Reset the number of recursion calls
    DPLL.num_calls = 0
    
    symbols = {lit if lit.sign else -lit for clause in KB for lit in clause}
    
    return(DPLL(KB, symbols, set(), debug, optim))


def test_DPLL():
    '''Test suit for the DPLL routines.'''
    A, B, C, D = (Literal(_) for _ in  "ABCD")
    
    # Return True if the model satisfies the KB, False otherwise
    assert DPLL([{A, B}], {A, B}, set())[0]
    assert DPLL([{A}], {A}, {B})[0]
    assert DPLL([{B}], {B}, {A})[0] 
    assert DPLL([{-B}], {B}, {-A})[0] 
    
    assert not DPLLSatisfiable([{A, B}, {A, -B}, {-A, B}, {-A, -B}])[0]
    assert DPLLSatisfiable([{A,B},{A,-C},{-A,B,D}])[0]

test_DPLL()

#### Exercise 7.20

The cells below show the conversion of the KB given in Russell and Norvig (2016) into CNF form. Finally, we test the satisfiability of the KB.

### S1
$$A \iff (B \vee E) $$
$$[A \Rightarrow (B \vee E)] \wedge [(B \vee E) \Rightarrow A] $$
$$(\neg A \vee B \vee E) \wedge [\neg(B \vee E) \vee A] $$
$$(\neg A \vee B \vee E) \wedge [(\neg B \wedge \neg E) \vee A] $$
$$(\neg A \vee B \vee E) \wedge (\neg B \vee A) \wedge (\neg E \vee A)$$

### S2
$$E \Rightarrow D$$
$$\neg E \vee D$$

### S3

$$C \wedge F \Rightarrow \neg B$$

$$\neg (C \wedge F) \vee \neg B$$

$$\neg C \vee \neg F \vee \neg B$$

### S4

$$E \Rightarrow B$$
$$\neg E \vee B$$

### S5

$$B \Rightarrow F$$
$$\neg B \vee F$$

### S6

$$B \Rightarrow C$$
$$\neg B \vee C$$

In [11]:
A, B, C, D, E, F = (Literal(_) for _ in  "ABCDEF")
s1 = [{-A, B, E}, {-B, A}, {-E, A}]
s2 = [{-E, D}]
s3 = [{-C, -F, -B}]
s4 = [{-E, B}]
s5 = [{-B, F}]
s6 = [{-B, C}]
KB = s1 + s2 + s3 + s4 + s5 + s6

print("KB: {}\n".format(KB))
print("Optimization is ON. KB Satisfiable? {}.\nModel: {}.\nNumber of DPLL calls: {}.\n".format(
      *DPLLSatisfiable(KB, debug=True, optim=True), DPLL.num_calls))
print("Optimization is OFF. KB Satisfiable? {}.\nModel: {}.\nNumber of DPLL calls: {}.\n".format(
      *DPLLSatisfiable(KB, debug=True, optim=False), DPLL.num_calls))

KB: [{-A, E, B}, {-B, A}, {-E, A}, {-E, D}, {-B, -F, -C}, {-E, B}, {-B, F}, {-B, C}]

[{-A, E, B}, {-B, A}, {-E, A}, {-E, D}, {-B, -F, -C}, {-E, B}, {-B, F}, {-B, C}] {D, B, A, C, F, E} set()
[{-A, E, B}, {-B, A}, {-E, A}, {-B, -C, -F}, {-E, B}, {-B, F}, {-B, C}] {B, A, C, F, E} {D}
[{A}, {-E, A}, {-F, -C}, {F}, {C}] {A, F, E, C} {D, B}
[{-C, -F}, {F}, {C}] {F, E, C} {A, D, B}
[{-F, -C}, {F}, {C}] {F, C} {A, -E, D, B}
[{-C}, {C}] {C} {-E, F, D, B, A}
[set()] set() {-E, F, D, B, A, -C}
[{-A, E}, {-E, A}, {-E}] {A, F, E, C} {-B, D}
[{-A, E}, {-E, A}, {-E}] {A, E, C} {-B, -F, D}
[{-A, E}, {-E, A}, {-E}] {A, E} {-B, -C, -F, D}
[{-A}] {A} {-E, -F, D, -B, -C}
[] set() {-E, -F, D, -B, -A, -C}
Optimization is ON. KB Satisfiable? True.
Model: {E: 'false', F: 'false', D: 'true', B: 'false', A: 'false', C: 'false'}.
Number of DPLL calls: 12.

[{-A, E, B}, {-B, A}, {-E, A}, {-E, D}, {-B, -F, -C}, {-E, B}, {-B, F}, {-B, C}] {D, B, A, C, F, E} set()
[{-A, E, B}, {-B, A}, {-E, A}, {-B, -C, -F}, {-E, 

We see that even in this very simple KB, the heuristics reduce the number of expansions considerably. Note, also, that they output different models.

# References

Hjelle, G. A. (2018). Primer on Python Decorators. Retrieved from https://realpython.com/primer-on-python-decorators/

Russell, S. J., & Norvig, P. (2016). _Artificial intelligence: a modern approach (3rd ed.)_. Boston: Pearson.